# cohttp_async_example

A simple example of use of the DuckDuckGo API server by `cohttp.async`.

In [1]:
#thread ;;
#require "yojson" ;;
#require "cohttp.top" ;;
#require "cohttp.async" ;;

/home/opam/.opam/4.04.1/lib/ocaml/threads: added to search path
/home/opam/.opam/4.04.1/lib/ocaml/unix.cma: loaded
/home/opam/.opam/4.04.1/lib/ocaml/threads/threads.cma: loaded
/home/opam/.opam/4.04.1/lib/easy-format: added to search path
/home/opam/.opam/4.04.1/lib/easy-format/easy_format.cmo: loaded
/home/opam/.opam/4.04.1/lib/biniou: added to search path
/home/opam/.opam/4.04.1/lib/biniou/biniou.cma: loaded
/home/opam/.opam/4.04.1/lib/yojson: added to search path
/home/opam/.opam/4.04.1/lib/yojson/yojson.cmo: loaded
/home/opam/.opam/4.04.1/lib/bytes: added to search path
/home/opam/.opam/4.04.1/lib/re: added to search path
/home/opam/.opam/4.04.1/lib/re/re.cma: loaded
/home/opam/.opam/4.04.1/lib/re/re_emacs.cma: loaded
/home/opam/.opam/4.04.1/lib/stringext: added to search path
/home/opam/.opam/4.04.1/lib/stringext/stringext.cma: loaded
/home/opam/.opam/4.04.1/lib/re/re_posix.cma: loaded
/home/opam/.opam/4.04.1/lib/ocaml/bigarray.cma: loaded
/home/opam/.opam/4.04.1/lib/sexplib/0: ad

In [2]:
open Core ;;
open Async ;;
open Cohttp ;;
open Cohttp_async ;;

In [3]:
Thread.create (fun () -> never_returns (Scheduler.go ())) () ;; (* Start the Async scheduler. *)

- : Core.Thread.t = <abstr>

We obtain the meaning of a given query from the DuckDuckGo API server.

[Yojson](https://github.com/mjambon/yojson) is a JSON serialization/deserialization library written in OCaml, and
[Yojson.Basic.Util](https://mjambon.github.io/mjambon2016/yojson-doc/Yojson.Basic.Util.html) contains utilities for parsing a JSON.

In [4]:
let search_query query =
  let base_uri = Uri.of_string "http://api.duckduckgo.com/?format=json" in
  let uri = Uri.add_query_param base_uri ("q", [query]) in
  Client.get uri >>= fun (resp, body) -> (* GET contents from a given uri *)
  assert (Response.status resp = `OK) ; (* Check HTTP response code *)
  Body.to_string body >>| fun body -> (* Receive contents *)
  let json = Yojson.Basic.from_string body in (* Parse a JSON *)
  (* Find field "Definition" or "Abstract" in a JSON *)
  let def = Yojson.Basic.Util.(json |> member "Definition" |> to_string_option) in
  let abs = Yojson.Basic.Util.(json |> member "Abstract" |> to_string_option) in
  match def with
  | None | Some "" -> abs
  | _ -> def

val search_query : string -> string option Async_kernel__Deferred.t = <fun>

`Thread_safe.block_on_async_exn` waits an Async instance and returns its result.

In [5]:
Thread_safe.block_on_async_exn (fun () -> search_query "ocaml") ;;

- : string option =
Some
 "OCaml, originally named Objective Caml, is the main implementation of the programming language Caml, created by Xavier Leroy, J\195\169r\195\180me Vouillon, Damien Doligez, Didier R\195\169my, Asc\195\161nder Su\195\161rez and others in 1996. A member of the ML language family, OCaml extends the core Caml language with object-oriented programming constructs."